# PaddleMD API tutorial

## System setup

We use the `moleculekit` library for reading the input topologies and starting coordinates

In [1]:
from moleculekit.molecule import Molecule
import os

testdir = "./test-data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # Reading the system topology
mol.read(os.path.join(testdir, "input.coor"))  # Reading the initial simulation coordinates
mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

Next we will load a forcefield file and use the above topology to extract the relevant parameters which will be used for the simulation

In [2]:
from paddlemd.forcefields.forcefield import ForceField
from paddlemd.parameters import Parameters
import paddle

precision = paddle.float32
# device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))
parameters = Parameters(ff, mol, precision=precision)

2022-06-05 11:10:39,119 - root - WARNING - Warning: importing 'simtk.openmm' is deprecated.  Import 'openmm' instead.


Now we can create a `System` object which will contain the state of the system during the simulation, including:
1. The current atom coordinates
1. The current box size
1. The current atom velocities
1. The current atom forces

In [3]:
from paddlemd.integrator import maxwell_boltzmann
from paddlemd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/math_op_patch.py:278: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


Lastly we will create a `Force` object which will be used to evaluate the potential on a given `System` state

In [4]:
from paddlemd.forces import Forces
bonded = ["bonds", "angles", "dihedrals", "impropers", "1-4"]
# bonded = ["dihedrals"]
# forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5)
forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5, terms=bonded)
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'bonds': 3.957756519317627, 'angles': 2.844571113586426, 'dihedrals': 10.57987117767334, 'impropers': 1.2417094707489014, '1-4': 0.0, 'external': 0.0}]
Tensor(shape=[1, 688, 3], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[[ 3.20194221 ,  1.40353131 ,  3.54008961 ],
         [-20.73697662, -18.31974411,  9.39755249 ],
         [ 1.86052597 ,  4.54450178 , -4.52474356 ],
         ...,
         [ 0.00015092 ,  0.00010121 ,  0.00010221 ],
         [-0.00021580 , -0.00019182 , -0.00024192 ],
         [ 0.00006488 ,  0.00009061 ,  0.00013972 ]]])


## Dynamics

For performing the dynamics we will create an `Integrator` object for integrating the time steps of the simulation as well as a `Wrapper` object for wrapping the system coordinates within the periodic cell

In [5]:
from paddlemd.integrator import Integrator
from paddlemd.wrapper import Wrapper

langevin_temperature = 300  # K
langevin_gamma = 0.1
timestep = 1  # fs

integrator = Integrator(system, forces, timestep, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None)

In [6]:
from paddlemd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # Minimize the system steps=500

Iter  Epot            fmax    
   0    18.623908    47.097681
   1    742.506645    930.048814
   2    15.581048    30.797948
   3    14.288228    19.221545
   4    12.361484    13.097783
   5    11.714102    35.064515
   6    11.014398    13.932185
   7    10.539819    10.034238
   8    10.278650    7.252243
   9    10.006041    11.000469
  10    9.805500    12.476327
  11    9.624334    6.099770
  12    9.494328    3.868678
  13    9.401134    3.453931
  14    9.359479    8.285483
  15    9.266424    2.640222
  16    9.237603    2.628917
  17    9.192625    2.343936
  18    9.184314    4.700428
  19    9.140040    2.537128
  20    9.128266    1.391213
  21    9.116088    1.307519
  22    9.096715    1.802672
  23    9.101178    7.084076
  24    9.082276    3.440598
  25    9.059691    1.205466
  26    9.046657    1.183203
  27    9.030496    1.425538
  28    9.015735    2.880708
  29    9.002313    1.476721
  30    8.982564    2.092492
  31    8.969266    2.705697
  32    8.957081   

Create a CSV file logger for the simulation which keeps track of the energies and temperature.

In [7]:
from paddlemd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')

Writing logs to  logs/monitor.csv


Now we can finally perform the full dynamics

In [8]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 1000 # 1000
output_period = 10
save_period = 100
traj = []

trajectoryout = "mytrajectory.npy"

iterator = tqdm(range(1, int(steps / output_period) + 1))
# print(f"iterator={iterator}")
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
#     currpos = system.pos.detach().cpu().numpy().copy()
#     currpos = system.pos.detach()
    currpos = system.pos
#     print(currpos.shape)
    traj.append(currpos)
#     print(len(traj) )
#     print(f"iterator={iterator}")
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=2))

    logger.write_row({'iter':i*output_period,'ns':FS2NS*i*output_period*timestep,'epot':Epot,'ekin':Ekin,'etot':Epot+Ekin,'T':T})

100%|██████████| 100/100 [02:54<00:00,  1.75s/it]


In [9]:
paddle.to_tensor([2,3])

Tensor(shape=[2], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       [2, 3])

In [10]:
system.pos.shape, system.box.shape

([1, 688, 3], [1, 3, 3])

In [11]:
wrapidx = paddle.to_tensor([0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21])

In [12]:
pos = system.pos
# import torch
# torchpos = torch.Tensor(pos.numpy())
# torchidx = torch.Tensor([0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, \
#                          18, 19, 20, 21]).int32()
# torchtmp = torchpos[:, torchidx]
# tmp = pos[:, wrapidx]
tmp = paddle.gather(pos, wrapidx, axis=1)
print(tmp.shape)
tmp1 = paddle.sum(tmp, axis=1)
print(tmp1)
# com = paddle.sum(pos[:, wrapidx], axis=1) / len(wrapidx)

[1, 22, 3]
Tensor(shape=[1, 3], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[256.92028809, 256.57214355, 253.62548828]])
